In [ ]:

    def get_parent_topk(self, query: str, query_id: int, topk: int = 100) -> Union[Dict[int, float], List[int]]:
        if self.parent_pred_path and osp.exists(self.parent_pred_path):
            csv = pd.read_csv(self.parent_pred_path)
            csv = csv[['query_id', 'pred_rank']]
            csv = csv[csv['query_id'] == query_id]
            if len(csv):
                pred_rank = eval(csv['pred_rank'].iloc[0])
                initial_score_dict = {node_id: 1. / (rank + 1) for rank, node_id in enumerate(pred_rank)}
                return initial_score_dict, pred_rank[:topk]

        initial_score_dict = self.parent_vss(query, query_id)
        node_ids = list(initial_score_dict.keys())
        node_scores = list(initial_score_dict.values())
        top_k_idx = torch.topk(torch.FloatTensor(node_scores),
                               min(topk, len(node_scores)),
                               dim=-1).indices.view(-1).tolist()

        vss_top_candidates = [node_ids[i] for i in top_k_idx]
        return initial_score_dict, vss_top_candidates


In [3]:
import stark_qa

In [26]:
import subprocess
import os.path as osp
from stark_qa.models import VSS
import torch

dataset = "prime"
emb_model = "text-embedding-ada-002"
emb_dir = "/dfs/project/kgrlm/multiagent_reward/emb"

if not osp.exists(emb_dir):
    subprocess.run(["python", "functional/emb_download.py", "--dataset", "prime", "--emb_dir", emb_dir])

node_emb_dir = osp.join(emb_dir, dataset, emb_model, "doc")
query_emb_dir = osp.join(emb_dir, dataset, emb_model, "query")
skb = stark_qa.load_skb(dataset)
vss = VSS(skb, query_emb_dir, node_emb_dir, emb_model=emb_model)

def get_vss_topk(query, query_id, k=5):
    initial_score_dict = vss(query, query_id)
    node_ids = list(initial_score_dict.keys())
    node_scores = list(initial_score_dict.values())
    top_k_idx = torch.topk(torch.FloatTensor(node_scores),
                            min(k, len(node_scores)),
                            dim=-1).indices.view(-1).tolist()

    vss_top_candidates = [node_ids[i] for i in top_k_idx]
    return vss_top_candidates

candidate_ids = get_vss_topk("", 1)

Loading from /dfs/user/shirwu/.cache/huggingface/hub/datasets--snap-stanford--stark/snapshots/88269e23e90587f99476c5dd74e235a0877e69be/skb/prime/processed!
Loading query embeddings from /dfs/project/kgrlm/multiagent_reward/emb/prime/text-embedding-ada-002/query/query_emb_dict.pt
Loaded candidate_emb_dict from /dfs/project/kgrlm/multiagent_reward/emb/prime/text-embedding-ada-002/doc/candidate_emb_dict.pt!


[14796, 15307, 20645, 14986, 15365]

In [35]:
from stark_qa.evaluator import  Evaluator
def eval_func(candidate_ids, candidate_scores, answer_ids, metrics=["mrr"]):
    candidate_score_dict = {_id: score for _id, score in zip(candidate_ids, candidate_scores)} 
    evaluator = Evaluator(skb.candidate_ids)
    return evaluator(candidate_score_dict, torch.LongTensor(answer_ids), metrics)

candidate_ids = [14796, 15307, 20645, 14986, 15365]
candidate_scores = [1,2,3,4,5]
answer_ids = [14796]
eval_func(candidate_ids, candidate_scores, answer_ids)

{'mrr': 0.20000000298023224}